In [11]:
from typing import List, Tuple
from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features
from helper.clfmodel_functions import tune_model, seq_feat_selection, multi_metric_cv, plot_multi_score_cv_results, forward_feat_selection_hypertuning
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.tree import DecisionTreeClassifier

### Loading the cleaned dataset

In [12]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

In [13]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
columns_to_exclude = ['sex', 'ability to speak english', 'gave birth this year']
# Encoding the features and target, and excluding some columns
X_encoded, y_encoded = encode_all_features(X, y, [])
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [14]:
X_train.head()

,age,education,workinghours,ability to speak english,sex_Female,sex_Male,occupation_Construction/Extraction,occupation_Counseling/Mental Health Services,occupation_Education,occupation_Entertainment,...,workclass_private,workclass_self employed,marital status_Divorced,marital status_Husband,marital status_Never married,marital status_Separated,marital status_Widowed,marital status_Wife,gave birth this year_No,gave birth this year_Yes
6317,22,16,36,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
740,61,22,40,1,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
3781,48,16,40,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
7850,62,18,65,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2963,53,19,44,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


### Model

In [15]:
# Decision Tree model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

dt_preds = dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_preds)

print(classification_report(y_test, dt_preds))
print("Decision Tree Accuracy:", dt_accuracy)

              precision    recall  f1-score   support

           0       0.78      0.79      0.78      1175
           1       0.59      0.57      0.58       625

    accuracy                           0.71      1800
   macro avg       0.68      0.68      0.68      1800
weighted avg       0.71      0.71      0.71      1800

Decision Tree Accuracy: 0.715


In [16]:
seq_feat_sel = False
if seq_feat_sel:
    seq_feat_selection(DecisionTreeClassifier(), X_train, y_train, direction='backward')

### Hyperparameter tuning

In [17]:
if False:
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': [None, 5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': [None, 'sqrt', 'log2'],
        'random_state': [42]
    }
    
    best_params, best_model, best_accuracy = tune_model(DecisionTreeClassifier(), X_train, y_train, X_test, y_test, param_grid)
    
    print("\nBest Hyperparameters:", best_params)
    print("Best Model:", best_model)
    print("Best Model Accuracy:", best_accuracy)

In [18]:
# Forward feature selection with hyperparameter tuning
if False:
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': [None, 5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': [None, 'sqrt', 'log2'],
        'random_state': [42]
    }
    
    best_subset, best_params, best_score = forward_feat_selection_hypertuning(DecisionTreeClassifier(), param_grid, X_train, y_train, X_test, y_test)
    
    print("Best subset of features:", best_subset)
    print("Best hyperparameters:", best_params)
    print("Best model accuracy:", best_score)
    
    # Use the best subset and best hyperparameters for final model
    final_model = DecisionTreeClassifier(**best_params)
    final_model.fit(X_train[best_subset], y_train)
    final_model_preds = final_model.predict(X_test[best_subset])
    final_model_accuracy = accuracy_score(y_test, final_model_preds)
    
    print(classification_report(y_test, final_model_preds))

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for AdaBoost
param_grid = {
    'algorithm': ['SAMME', 'SAMME.R'],
    'n_estimators': [50, 75, 100, 150],
    'learning_rate': [0.1, 0.4, 0.5, 1.0],
    'estimator': [DecisionTreeClassifier(criterion= 'gini', max_depth= 10, max_features= None, min_samples_leaf= 2, min_samples_split= 10, random_state=42, splitter= 'random'), None],
    'random_state': [42]
}

best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train, y_train, X_test, y_test, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent


Best Hyperparameters: {'algorithm': 'SAMME.R', 'estimator': None, 'learning_rate': 0.4, 'n_estimators': 150, 'random_state': 42}
Best Model: AdaBoostClassifier(learning_rate=0.4, n_estimators=150, random_state=42)
Best Model Accuracy: 0.7927777777777778


### Saving the model

In [20]:
# save model
save_model(dt_model, '../output/saved_models/decision_tree_model.joblib')